**LIBRERIAS**

In [1]:
import pandas as pd
import json
import ast
import gzip
import numpy as np

1.0 **APERTURAS DE DATOS**

1.1 *REVIEWS*

In [282]:
users_reviews = []

with gzip.open('user_reviews.json.gz', 'rt', encoding='utf-8') as f:
    for line in f:
        try:
            user_item = ast.literal_eval(line)
            users_reviews.append(user_item)
        except (SyntaxError, ValueError) as e:
            print(f"Error evaluating line as literal: {e}")
#4.5seg / 4.1

1.2 *GAMES*

In [2]:
steam_games_list = []
with open('output_steam_games.json', 'r', encoding='MacRoman') as f:
    for line in f:
        try:
            game_data = json.loads(line)
            steam_games_list.append(game_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
#17.7 seg / 2m 44.3 / 24.4s

1.3 *ITEMS*

In [3]:
users_items = []

with gzip.open('users_items.json.gz', 'rt', encoding='utf-8') as f:
    for line in f:
        try:
            user_item_line = ast.literal_eval(line)
            users_items.append(user_item_line)
        except (SyntaxError, ValueError) as e:
            print(f"Error evaluating line as literal: {e}")
#3min 37seg / 5m 52.9 / 3m 38.5

2.0 **NORMALIZACION**

2.1 *REVIEWS*

In [283]:
df_reviews = pd.json_normalize(
    users_reviews, 'reviews',['user_id', 'user_url'])
# 8.2 / 16.5

In [285]:
df_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637


2.2 *GAMES*

2.2.1 *LIMPIEZA DE DATOS EN EL DATA FRAME*

In [4]:
df_games_sucio = pd.json_normalize(steam_games_list)
#2.1

In [5]:
#Buscamos eliminar las columnas que tienen solamente NaN, utilizaremos una mascara
colums_nan = ['publisher','genres','app_name','title','url','release_date','tags','reviews_url','discount_price','specs','price','early_access','id','metascore','developer']
#0.1

In [6]:
#Creamos la mascara (el filtro)
mascara = df_games_sucio.index <= 88309
#0.2

In [7]:
# Transcribimos los datos que queremos utilizando la mascara
df_games_anidado = df_games_sucio[~mascara | df_games_sucio[colums_nan].notna().all(axis=1)]
#0.2

In [12]:
#Ahora desanidaremos las columnas que estan anidadas en el df, asi nos queda limpio
df_games = df_games_anidado.explode('genres')
#df_games = df_games.explode('tags')
#df_games = df_games.explode('specs')
#1m 15.2

2.3 *ITEMS*

In [2]:
df_items = pd.json_normalize(
    users_items, 'items', ['user_id', 'items_count', 'steam_id', 'user_url']
)
#2m 45.8 / 3m 46.4

NameError: name 'pd' is not defined

In [105]:
df_items = pd.json_normalize(
    users_items, 'items', ['user_id']
)

In [104]:
del df_items

In [1]:
df_reviews.head()

NameError: name 'df_reviews' is not defined

In [24]:
df_items.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [14]:
#creamos copias por seguridad
df_reviews_1 = df_reviews.copy()
df_games_1 = df_games.copy()
df_items_1 = df_items.copy()

3.0  **ELIMINAR COLUMNAS INECESARIAS**

3.1 *REVIEWS*

In [290]:
colums_a_eliminar_reviews = ['user_url','funny','helpful']
df_reviews_1 = df_reviews.drop(columns = colums_a_eliminar_reviews, axis = 1)

3.2 *GAMES*

In [142]:
df_games_1 = df_games_anidado.drop(columns = ['publisher','url','discount_price','price','early_access','tags','reviews_url','specs','app_name','title','metascore','developer','user_id','steam_id','items','items_count'], axis = 1)

In [56]:
#aqui elimino la fila de un id que todos sus valores son NaN y los NaN de Id
mascara_id_nan = df_games_1['id'] == '773570'
df_games_1 = df_games_1[~(mascara_id_nan | df_games_1['id'].isna())]

3281 missing en genres

In [58]:
#aqui elimino los valores NaN de Genres
df_games_1 = df_games_1.dropna(subset=['genres'])

In [89]:
# Crear una máscara booleana para identificar las filas con valores nulos
filas_con_nulos = df_games_1.isna().any(axis=1)
# Filtrar el DataFrame original para ver las filas con valores nulos
df_con_nulos = df_games_1[filas_con_nulos]
df_con_nulos

,genres,app_name,title,release_date,id
90890,"[Action, Indie]",NaN,NaN,2014-08-26,317160


In [78]:
#Necesito llenar los valores vacios de fecha, para esto voy a utilizar la moda, ya que las probabilidades de que este juego haya salido en la fecha mas recurrente es más alta.
#Primero visualizamos la metrica para esta columna
df_games_1['release_date'].mode()

0    2012-10-16
Name: release_date, dtype: object

In [88]:
#ahora aplicamos hacemos el reemplazo en aquellos valores null
mode_data = df_games_1['release_date'].mode().iloc[0]
df_games_1['release_date'].fillna(mode_data, inplace=True)

In [9]:
#como no lo esta desanidando entonces voy a agregar un paso más, voy a convertir las cadenas en listas utilizando ast.literal_eval
df_games_1['genres'] = df_games_1['genres'].apply(ast.literal_eval)

In [11]:
#vamos a ver que type nos toma despues de hacer el comando anterior
type(df_games_1['genres'][0])
#ahora ya podremos hacer el explode(), antes lo tomaba como str

list

In [12]:
#Ahora desanidaremos las columnas que estan anidadas en el df en la columna de genres, asi nos queda limpio
df_games_1 = df_games_1.explode('genres')

In [14]:
#cambiamos el nombre de la columna id de games por item_id, ya que todos estan identificados de esta forma
df_games_1.rename(columns={'id': 'item_id'}, inplace=True)

3.3 *ITEMS*

In [276]:
df_items_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3285246 entries, 0 to 3285245
Data columns (total 3 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int64 
 1   playtime_forever  int64 
 2   user_id           object
dtypes: int64(2), object(1)
memory usage: 75.2+ MB


In [275]:
#columnas a eliminar, que no necesitaremos
columnas_a_eliminar = ['item_name','playtime_2weeks']
df_items_1 = df_items_1.drop(columns = columnas_a_eliminar, axis = 1)

In [278]:
#eliminamos las filas con 0 time jugado en items
df_items_1 = df_items_1[df_items_1['playtime_forever'] != 0]

**PASAR MIS DF A ARCHIVOS CSV PARA QUE SEA MAS OPTIMO**

In [20]:
df_reviews_1.to_csv('reviews.csv', index=False)
df_games_4.to_csv('games_complemento_ml.csv', index=False)
df_items_1.to_csv('items_v4.0_con_3_columnas.csv', index=False)

In [23]:
df_reviews_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   posted       59305 non-null  object
 1   last_edited  59305 non-null  object
 2   item_id      59305 non-null  object
 3   helpful      59305 non-null  object
 4   recommend    59305 non-null  bool  
 5   review       59305 non-null  object
 6   user_id      59305 non-null  object
dtypes: bool(1), object(6)
memory usage: 2.8+ MB


In [28]:
df_games_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71552 entries, 0 to 28850
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        71552 non-null  object
 1   release_date  71552 non-null  object
 2   item_id       71552 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [39]:
df_games_1['release_date'].iloc[36]

'Soon..'

In [269]:
df_items_1.head()

,item_id,playtime_forever
0,10,6
1,30,7
2,300,4733
3,240,1853
4,3830,333


In [136]:
df_items_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3285246 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   user_id           object
dtypes: int64(1), object(3)
memory usage: 125.3+ MB


**ETL GAMES**

*VAMOS A NORMALIZAR FECHA*

In [229]:
df_games_2 = df_games_1.copy()

In [33]:
#verificampos cual es el tipo de dato que esta almacenado en la columna de release_date
type(df_games_2['release_date'][0])

pandas.core.series.Series

In [245]:
#Verificamos cuantos valores malos hay
import re
from collections import defaultdict

# Lista de strings a buscar
strings_a_buscar = [
    'Jun 2009', 'Oct 2010', 'Feb 2011', 'Aug 2014', 'Sep 2014', 'Apr 2015', 'Apr 2016', 'Jul 2016', 'June 2016', 'Q2 2017',
    'Q2 2018', 'Winter 2017', 'Mar 2018', '14 July', 'Jul 2017', 'Summer 2017', 'Spring 2018', 'Winter 2018', 'October 2017',
    'Fall 2017', 'Nov 2017', 'Q1 2018', 'Dec 2017', 'Январь 2018', 'First quarter of 2018', 'H2 2018', "21 Jun, 2017",
    '2018年初頭発売予定', 'Feb 2018', 'August 2017', '2018 [Now get free Pre Alpha]', '1st Quarter 2018', '2018年1月',
    'Fall 2018', 'Spring 2017', 'January 2019', 'Q4 2017', 'Early 2018', 'Q2 2018 (Tentative)', 'Coming 2017',
    '2017 Q4', 'August 2018', 'Early Spring 2018', 'TBA 2017', 'Aug 2017', 'Coming Q1 2018', 'December 2017',
    'Coming Q3 2017', 'Early 2017', 'End 2017', 'Q1 2017', 'Jan 2017', 'January 2017', "When it's done (2017)",
    "Q1 (ish) 2017", 'Nov 2016', 'Oct 2016', 'Jun 2016', 'Jun 2015', 'May 2015', 'Feb 2015', 'Jan 2015', 'Nov 2014',
    'Jul 2014', 'May 2014', 'Feb 2013', 'Dec 2012', 'Jul 2010', 'Mar 2010', 'Jan 2010', 'Oct 2009', 'Sep 2009',
    'Late 2016 - Early 2017', 'Coming late 2017', 'Hitting Early Access in 2017', '2018 early', "Q1, 2018", 'January 2018'
]

# Función para contar las ocurrencias de las palabras en 'release_date' y devolver un diccionario
def contar_ocurrencias_strings(df, strings):
    # Crear un diccionario para almacenar el recuento de cada palabra
    recuento_palabras = defaultdict(int)

    # Crear un patrón de expresión regular con las cadenas a buscar
    patron = '|'.join(re.escape(s) for s in strings)

    # Utilizar str.contains() para buscar las ocurrencias en 'release_date'
    ocurrencias = df['release_date'].str.extractall(f'({patron})').groupby(0).size()

    # Convertir el resultado en un diccionario
    recuento_palabras.update(ocurrencias.to_dict())

    return recuento_palabras

# Llamar a la función para contar las ocurrencias
recuento = contar_ocurrencias_strings(df_games_2, strings_a_buscar)
data = {
    'Fecha_mal_tipeada' : list(recuento.keys()),
    'Cantidad' : list(recuento.values())
}
df_recuento = pd.DataFrame(data)
df_recuento.head()

,Fecha_mal_tipeada,Cantidad


In [244]:
diccionario_limpiado = {
    'Jun 2009': '2009-06-01',
    'Oct 2010': '2010-10-01',
    'Feb 2011': '2011-02-01',
    'Aug 2014': '2014-08-01',
    'Sep 2014': '2014-09-01',
    'Apr 2015': '2015-04-01',
    'Apr 2016': '2016-04-01',
    'Jul 2016': '2016-07-01',
    'June 2016': '2016-06-01',
    'Q2 2017': '2017-02-16',
    'Q2 2018': '2018-02-16',
    'Winter 2017': '2017-01-01',
    'Mar 2018': '2018-03-01',
    '14 July': '2017-07-01',
    'Jul 2017': '2017-07-01',
    'Summer 2017': '2017-07-01',
    'Spring 2018': '2018-04-01',
    'Winter 2018': '2018-01-01',
    'October 2017': '2017-10-01',
    'Fall 2017': '2017-09-01',
    'Nov 2017': '2017-11-01',
    'Q1 2018': '2018-01-16',
    'Dec 2017': '2017-12-01',
    'Январь 2018': '2018-01-01',
    'First quarter of 2018': '2018-01-16',
    'H2 2018': '2018-07-16',
    "21 Jun, 2017": '2017-06-21',
    '2018年初頭発売予定': '2018-01-01',
    'Feb 2018': '2018-02-01',
    'August 2017': '2017-08-01',
    '2018 [Now get free Pre Alpha]': '2018-01-01',
    '1st Quarter 2018': '2018-01-16',
    '2018年1月': '2018-01-01',
    'Fall 2018': '2018-09-01',
    'Spring 2017': '2017-04-01',
    'January 2019': '2019-01-01',
    'Q4 2017': '2017-04-16',
    'Early 2018': '2018-01-01',
    'Q2 2018 (Tentative)': '2018-02-16',
    'Coming 2017': '2017-01-01',
    '2017 Q4': '2017-04-01',
    'August 2018': '2018-08-01',
    'Early Spring 2018': '2018-03-01',
    'TBA 2017': '2017-01-01',
    'Aug 2017': '2017-08-01',
    'Coming Q1 2018': '2018-01-01',
    'December 2017': '2017-12-01',
    'Coming Q3 2017': '2017-03-01',
    'Early 2017': '2017-01-01',
    'End 2017': '2017-12-01',
    'Q1 2017': '2017-01-16',
    'Jan 2017': '2017-01-01',
    'January 2017': '2017-01-01',
    "When it's done (2017)": '2017-01-01',
    "Q1 (ish) 2017": '2017-01-16',
    'Nov 2016': '2016-11-01',
    'Oct 2016': '2016-10-01',
    'Jun 2016': '2016-06-01',
    'Jun 2015': '2015-06-01',
    'May 2015': '2015-05-01',
    'Feb 2015': '2015-02-01',
    'Jan 2015': '2015-01-01',
    'Nov 2014': '2014-11-01',
    'Jul 2014': '2014-07-01',
    'May 2014': '2014-05-01',
    'Feb 2013': '2013-02-01',
    'Dec 2012': '2012-12-01',
    'Jul 2010': '2010-07-01',
    'Mar 2010': '2010-03-01',
    'Jan 2010': '2010-01-01',
    'Oct 2009': '2009-10-01',
    'Sep 2009': '2009-09-01',
    'Late 2016 - Early 2017': '2017-01-01',
    'Coming late 2017': '2017-12-01',
    'Hitting Early Access in 2017': '2017-01-01',
    '2018 early': '2018-01-01',
    "Q1, 2018": '2018-02-16',
    'January 2018': '2018-01-01',
    'Fall 2017' : '2017-09-01',
    'Coming Fall 2017':'2017-09-01',
    'Aug 2015': '2015-08-01',
    'Q1 (ish), 2017': '2017-01-16',
    '2017 Q1': '2017-01-16',
    '2017 Q1':'2017-01-16',
    'Apr 2017':'2017-04-01',
    'Oct 2017':'2017-10-01',
    'Jan 2018': '2018-01-01',
    '15.01.2018' : '2018-01-15'
}

# Reemplazar los valores en la columna 'release_date'
df_games_2['release_date'] = df_games_2['release_date'].replace(diccionario_limpiado)

In [236]:
df_games_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71552 entries, 0 to 28850
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        71552 non-null  object
 1   release_date  71552 non-null  object
 2   item_id       71552 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.2+ MB


In [237]:
#Antes que nada transformamos los valores de 'release_date' que solamente tengan año, es decir, un formato 'YYYY', para agregarle un '-01-01'. Como solo tiene el año podemos deducir que lo hicieron el primer dia del primer mes por eso no se coloco el resto, o tal vez fue un error de tipeo pero, vamos a hacerlo de esta manera para evitar que a la hora de cambiarlo de tipo de valor no tengamos valores vacios.
#df_games_2 = df_games_2[df_games_2['release_date'].str.match(r'\d{4}')]
df_games_2['release_date'] = df_games_2['release_date'].str.replace(r'^(\d{4})$', r'\1-01-01', regex=True)

In [238]:
df_games_2.head()

,genres,release_date,item_id
0,Action,2018-01-04,761140
0,Casual,2018-01-04,761140
0,Indie,2018-01-04,761140
0,Simulation,2018-01-04,761140
0,Strategy,2018-01-04,761140


In [246]:
# Hay valores no validos dentro de esta columna, como por ejemplo 'Soon...', y estos tipos de valores no me permiten aplicar un pd.to_datatime(), entonces lo que haré será filtrar las filas con valores de fecha válidos o solo el año
df_games_3 = df_games_2[df_games_2['release_date'].str.match(r'\d{4}(-\d{2}-\d{2})?')]

In [247]:
#Hay 287 datos con un formato incorrecto
df_games_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71265 entries, 0 to 28850
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        71265 non-null  object
 1   release_date  71265 non-null  object
 2   item_id       71265 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


*Todos estos datos no tienen ninguna fecha, lo unico que dice es que saldran muy pronto, por ende, lo vamos a descartar pero queda resgistrado cuales son y se pueden visualizar*

In [248]:
#quiero guardar en df esos valores incorrectos para poder verlos y poder hacer un cambio
mascara_formato_incorrecto = ~df_games_2['release_date'].str.match(r'^\d{4}(-\d{2}-\d{2})?$')
df_formato_incorrecto = df_games_2[mascara_formato_incorrecto]

In [249]:
df_formato_incorrecto.head()

,genres,release_date,item_id
9,Action,Soon..,768570
9,Indie,Soon..,768570
9,Simulation,Soon..,768570
9,Early Access,Soon..,768570
36,Adventure,Beta测试已开启,714020


In [268]:
#exportamos el archivo
df_formato_incorrecto.to_csv('games_sin_fecha_ConIndex.csv',index=True)

In [250]:
# Convertirmos 'release_date' a formato datetime
# 'errors='coerce'' permite que la conversión continúe incluso si algunas filas contienen valores que no son fechas válidas. Cuando se encuentra una cadena no válida, esa fila obtiene un valor NaT en lugar de una fecha.
df_games_3['release_date'] = pd.to_datetime(df_games_3['release_date'], errors='coerce')

C:\Users\PC\AppData\Local\Temp\ipykernel_10752\4254015632.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_3['release_date'] = pd.to_datetime(df_games_3['release_date'], errors='coerce')


In [253]:
df_games_3.head()

,genres,release_date,item_id
0,Action,2018-01-04,761140
0,Casual,2018-01-04,761140
0,Indie,2018-01-04,761140
0,Simulation,2018-01-04,761140
0,Strategy,2018-01-04,761140


In [258]:
df_games_3['year'] = df_games_3['release_date'].dt.year
df_games_3 = df_games_3[['item_id','genres','year','release_date']]

C:\Users\PC\AppData\Local\Temp\ipykernel_10752\1008890983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_3['year'] = df_games_3['release_date'].dt.year


In [261]:
#correccion en error de tipeo en df_games
correccion = {
    'Animation &amp; Modeling':'Animation and Modeling',
    'Design &amp; Illustration': 'Design and Illustration'
}
# regex=True permite que los valores se reemplacen utilizando expresiones regulares, lo que garantiza que incluso si hay otros caracteres en los valores, se realizará el reemplazo correcto.
df_games_3['genres'] = df_games_3['genres'].replace(correccion, regex=True)

In [638]:
df_games_3.rename(columns={'year':'year_release'},inplace=True)

In [639]:
df_games_3.head()

,item_id,genres,year_release,release_date
0,761140,Action,2018,2018-01-04
0,761140,Casual,2018,2018-01-04
0,761140,Indie,2018,2018-01-04
0,761140,Simulation,2018,2018-01-04
0,761140,Strategy,2018,2018-01-04


**ETL REVIEWS**

In [303]:
df_reviews_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   posted       59305 non-null  object
 1   last_edited  59305 non-null  object
 2   item_id      59305 non-null  object
 3   recommend    59305 non-null  bool  
 4   review       59305 non-null  object
 5   user_id      59305 non-null  object
dtypes: bool(1), object(5)
memory usage: 2.3+ MB


In [295]:
# Limpiamos la columna 'posted', le vamos a sacar la palabras Posted, ya que es redundante, tambien sacaremos los puntos finales
df_reviews_1['posted'] = df_reviews_1['posted'].str.replace('Posted ', '')
df_reviews_1['posted'] = df_reviews_1['posted'].str.replace('.', '')

In [307]:
#Tambien tenemos que hacer esta limpieza en la columna 'last_edited'
df_reviews_1['last_edited'] = df_reviews_1['last_edited'].str.replace('Last edited ', '')
df_reviews_1['last_edited'] = df_reviews_1['last_edited'].str.replace('.', '')

In [305]:
df_reviews_1.head(10)

,posted,last_edited,item_id,recommend,review,user_id
0,"November 5, 2011",,1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,"July 15, 2011",,22200,True,It's unique and worth a playthrough.,76561197970982479
2,"April 21, 2011",,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,"June 24, 2014",,251610,True,I know what you think when you see this title ...,js41637
4,"September 8, 2013",,227300,True,For a simple (it's actually not all that simpl...,js41637
5,"November 29, 2013",,239030,True,Very fun little game to play when your bored o...,js41637
6,February 3,,248820,True,A suitably punishing roguelike platformer. Wi...,evcentric
7,"December 4, 2015","Last edited December 5, 2015.",370360,True,"""Run for fun? What the hell kind of fun is that?""",evcentric
8,"November 3, 2014",,237930,True,"Elegant integration of gameplay, story, world ...",evcentric
9,"October 15, 2014",,263360,True,"Random drops and random quests, with stat poin...",evcentric


In [456]:
df_reviews_2=df_reviews_1.copy()

In [455]:
#A las fechas 'Febraury 29' le añadiremos el año 2016, ya que ese año si era un año bisiesto.
df_reviews_1['posted'] = df_reviews_1['posted'].str.replace('February 29', 'February 29, 2016')

In [457]:
from datetime import datetime

# Define una función para convertir fechas en objetos datetime
def parse_date(date_str):
    # Intenta analizar la fecha en diferentes formatos
    date_formats = ['%B %d, %Y', '%B %d']
    
    for date_format in date_formats:
        try:
            return datetime.strptime(date_str, date_format)
        except ValueError:
            continue
    
    # Si ninguno de los formatos coincide, devuelve None
    return date_str

In [458]:
df_reviews_2['posted'] = df_reviews_2['posted'].apply(parse_date)
df_reviews_2['last_edited'] = df_reviews_2['last_edited'].apply(parse_date)

In [374]:
#Vamos a extrer los valores que se realizo el cambio, esto lo vamos a saber porque el type de estos valores no sera datatime, sino str
def busqueda(date_column):
    lista = []
    for idx, date_str in date_column.items():
        try:
            # Intenta analizar la fecha
            parsed_date = pd.to_datetime(date_str, errors='coerce')
            if pd.notna(parsed_date):
                # La fecha se pudo analizar, continúa al siguiente elemento
                continue
        except:
            pass
        # Si no se pudo analizar la fecha o hay otro tipo de error, agrega el elemento a la lista
        lista.append((idx, date_str))
    return lista

In [500]:
valores_no_convertidos = busqueda(df_sin_nulos['posted'])

In [377]:
valores_no_convertidos_last_edited = busqueda(df_reviews_2['last_edited'])

In [376]:
valores_no_convertidos

[(843, 'February 29'),
 (3815, 'February 29'),
 (6835, 'February 29'),
 (7428, 'February 29'),
 (9902, 'February 29'),
 (13087, 'February 29'),
 (15481, 'February 29'),
 (15648, 'February 29'),
 (23571, 'February 29'),
 (26882, 'February 29'),
 (27018, 'February 29'),
 (27323, 'February 29'),
 (27364, 'February 29'),
 (29112, 'February 29'),
 (30426, 'February 29'),
 (37392, 'February 29'),
 (38983, 'February 29'),
 (40322, 'February 29'),
 (40388, 'February 29'),
 (41101, 'February 29'),
 (49410, 'February 29'),
 (49411, 'February 29'),
 (52326, 'February 29'),
 (56697, 'February 29'),
 (57240, 'February 29')]

In [465]:
df_reviews_2.head()

,posted,last_edited,item_id,recommend,review,user_id
0,2011-11-05 00:00:00,,1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,2011-07-15 00:00:00,,22200,True,It's unique and worth a playthrough.,76561197970982479
2,2011-04-21 00:00:00,,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,2014-06-24 00:00:00,,251610,True,I know what you think when you see this title ...,js41637
4,2013-09-08 00:00:00,,227300,True,For a simple (it's actually not all that simpl...,js41637


In [ ]:
valores_no_convertidos_v2 = busqueda(df_sin_nulos['posted'])

In [462]:
#todos los valores estan convertidos de la columna 'posted'
valores_no_convertidos_v2

[]

*crearemos una funcion para tomar los valores de la columna 'last_edited' o de 'posted' en caso del tipo de dato*

hay un inconveniente, en la columna 'posted' hay muchos valores que tienen mes, dia y año, pero, hay otros que no tienen el año, solamente mes dia y nos sirve el cambio si es que en la columna 'last_edited' hay valores con el formato que queres (mes, dia y año), pero hay otras ocaciones en donde pasa algo particular, en la columna 'posted' hay mes, dia y año, pero en 'last_edited' hay mes y dia, y si tomamos el criterio de agarrar lo que hay en 'last_edited' por encima de todo vamos a tener errores.

Entonces la solución que busque si 'last_edited' hay algo, si no hay nada ponemos lo de 'posted' este como este, pero si en 'last_edited' hay algo, que primero me busque si 'last_edited' tiene año, si 'last_edited' tiene año ponemos lo que hay en 'last_edited' sino, nos fijamos si en 'posted' hay año, si en 'posted' hay año, ponemos el 'año' de posted y el mes y dia de 'last_edited'. En caso de que en 'last_edited' haya valores, pero no año y en 'posted' tampoco haya año, ponemos lo que haya en 'last_edited'

In [468]:
#Vemos que es lo que hay en dentro de posted
df_reviews_2['posted'].iloc[843]

datetime.datetime(2016, 2, 29, 0, 0)

In [473]:
#verificamos como es que obtenemos solo el año
df_reviews_2['posted'][842].year

1900

In [490]:
#hacemos copia por seguridad
df_sin_nulos = df_reviews_2.copy()

In [496]:
df_sin_nulos.head()

,posted,last_edited,item_id,recommend,review,user_id
0,2011-11-05 00:00:00,<NA>,1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,2011-07-15 00:00:00,<NA>,22200,True,It's unique and worth a playthrough.,76561197970982479
2,2011-04-21 00:00:00,<NA>,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,2014-06-24 00:00:00,<NA>,251610,True,I know what you think when you see this title ...,js41637
4,2013-09-08 00:00:00,<NA>,227300,True,For a simple (it's actually not all that simpl...,js41637


In [491]:
#vamos a ver la moda de los valores distintos a '', para poder plicar ese año a las dechas que son del año 1900 ya que ese año se relleno automaticamente por la funcion datatime
df_sin_nulos['last_edited'] = df_reviews_2['last_edited'].replace('', pd.NA)

In [495]:
#La moda de la columna 'last_edited' es del año 2013, podria hacer una opcion para reemplazar por el año 1900, pero antes, tendremos en cuenta si en la columna 'posted' tiene año y utilizaremos ese año, en caso de que no utlizaremos el año de la moda
df_sin_nulos.describe()

,posted,last_edited,item_id,recommend,review,user_id
count,59305,6140,59305,59305,59305,59305
unique,1906,1014,3682,2,55313,25458
top,2014-06-21 00:00:00,2013-11-25 00:00:00,730,True,good game,76561198094665607
freq,225,99,3759,52473,100,20


In [519]:
#pasamos las columnas a el formato fecha y hora
df_sin_nulos['posted'] = pd.to_datetime(df_sin_nulos['posted'], errors='coerce')
df_sin_nulos['last_edited'] = pd.to_datetime(df_sin_nulos['last_edited'], errors='coerce')

In [531]:
#Los valores con NaT me estan dando prblemas para aplicar mi funcion, cambiaremos los NaT a Null
df_sin_nulos['last_edited'].fillna(np.nan, inplace=True)
df_sin_nulos['posted'].fillna(np.nan, inplace=True)

In [591]:
#Definimos la funcion que nos va a retornar la fecha correcta segun el analisis que hicimos
def fecha_final(fila):
    # Si 'last_edited' no es nulo y su año no es 1900, devuelve 'last_edited'
    if not pd.isnull(fila['last_edited']) and fila['last_edited'].year != 1900:
        return fila['last_edited']

    # Si 'last_edited' no es nulo, pero su año es 1900
    elif not pd.isnull(fila['last_edited']) and fila['last_edited'].year == 1900:
        # Si 'posted' no es nulo y su año no es 1900, combina el año de 'last_edited' con el mes y día de 'posted'
        if not pd.isnull(fila['posted']) and fila['posted'].year != 1900:
            fecha_modificada = pd.Timestamp(year=fila['posted'].year, month=fila['last_edited'].month, day=fila['last_edited'].day)
        # Si 'posted' también tiene año 1900, usa 2013 como el año y combina el mes y día de 'last_edited'
        elif not pd.isnull(fila['posted']) and fila['posted'].year == 1900:
            fecha_modificada = pd.Timestamp(year=2013, month=fila['last_edited'].month, day=fila['last_edited'].day)
            return fecha_modificada
        # Si no hay información en 'posted', simplemente devuelve 'last_edited'
        else:
            return fila['last_edited']
    
    # Si 'last_edited' es nulo, pero 'posted' no lo es
    elif not pd.isnull(fila['posted']):
        # Si 'posted' tiene año 1900, usa 2014 como el año y combina el mes y día de 'posted'
        if fila['posted'].year == 1900:
            fecha_modificada = pd.Timestamp(year=2014, month=fila['posted'].month, day=fila['posted'].day)
            return fecha_modificada
        # Si 'posted' no tiene año 1900, simplemente devuelve 'posted'
        else:
            return fila['posted']
    
    # Si tanto 'last_edited' como 'posted' son nulos, devuelve NaN
    else:
        return pd.NA

In [566]:
#creamos una copia para aplicarle las funciones y que no pierdamos el procesos anterior
df_sin_nulos_2 = df_sin_nulos.copy()

In [611]:
#creamo una nueva columna donde vamos a poner los valores de la fechas con corrección
df_sin_nulos_2['fecha'] = df_sin_nulos_2.apply(fecha_final,axis=1)

In [628]:
df_sin_nulos_2.head()

,posted,last_edited,item_id,recommend,review,user_id,date,year
0,2011-11-05,NaT,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011-11-05,2011
1,2011-07-15,NaT,22200,True,It's unique and worth a playthrough.,76561197970982479,2011-07-15,2011
2,2011-04-21,NaT,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011-04-21,2011
3,2014-06-24,NaT,251610,True,I know what you think when you see this title ...,js41637,2014-06-24,2014
4,2013-09-08,NaT,227300,True,For a simple (it's actually not all that simpl...,js41637,2013-09-08,2013


In [613]:
#definimos una funcion de relleno
def rellenamos(fila):
    if pd.isnull(fila['fecha']):
        fecha_modificada = pd.Timestamp(year=fila['posted'].year, month=fila['last_edited'].month, day=fila['last_edited'].day)
        return fecha_modificada
    else:
        return fila['fecha']

In [614]:
df_sin_nulos_2['fecha']=df_sin_nulos_2.apply(rellenamos, axis=1)

In [310]:
#Hay 10119 fechas que no tienen el formato año dia mes
df_reviews_2.head()

,posted,last_edited,item_id,recommend,review,user_id
0,"November 5, 2011",,1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,"July 15, 2011",,22200,True,It's unique and worth a playthrough.,76561197970982479
2,"April 21, 2011",,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,"June 24, 2014",,251610,True,I know what you think when you see this title ...,js41637
4,"September 8, 2013",,227300,True,For a simple (it's actually not all that simpl...,js41637


In [583]:
#buscamos los indices de esos valores nulos
indice_valores_null = df_sin_nulos_2[df_sin_nulos_2['fecha'].isna()].index

In [598]:
indice_valores_null

Index([   87,    88,   193,   214,   359,   417,   445,   477,   710,   715,
       ...
       58533, 58601, 58661, 58679, 58735, 58767, 58979, 59074, 59144, 59196],
      dtype='int64', length=860)

In [626]:
#agregaremos una columna de año y corregimos el nombre de año
df_sin_nulos_2.rename(columns={'fecha': 'date'}, inplace=True)
df_sin_nulos_2['year'] = df_sin_nulos_2['date'].dt.year

In [627]:
df_sin_nulos_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   posted       59305 non-null  datetime64[ns]
 1   last_edited  6136 non-null   datetime64[ns]
 2   item_id      59305 non-null  object        
 3   recommend    59305 non-null  bool          
 4   review       59305 non-null  object        
 5   user_id      59305 non-null  object        
 6   date         59305 non-null  datetime64[ns]
 7   year         59305 non-null  int32         
dtypes: bool(1), datetime64[ns](3), int32(1), object(3)
memory usage: 3.0+ MB


In [629]:
#guardamos el nuevo df eliminando las anteriores columnas de fechas
df_reviews_optimo = df_sin_nulos_2[['date','year','item_id','user_id','recommend','review']]

In [633]:
df_reviews_optimo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       59305 non-null  datetime64[ns]
 1   year       59305 non-null  int32         
 2   item_id    59305 non-null  object        
 3   user_id    59305 non-null  object        
 4   recommend  59305 non-null  bool          
 5   review     59305 non-null  object        
dtypes: bool(1), datetime64[ns](1), int32(1), object(3)
memory usage: 2.1+ MB


*exporto el df reviews a csv*

In [634]:
df_reviews_optimo.to_csv('reviews_v2.0_fechas_limpias.csv', index=False)